In [20]:
!pip uninstall protobuf -y

Found existing installation: protobuf 3.19.4
Uninstalling protobuf-3.19.4:
  Successfully uninstalled protobuf-3.19.4


In [21]:
!pip install protobuf==3.20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.5 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.0 which is incompatible.


In [8]:
!pip install --upgrade protobuf

  Using cached protobuf-4.21.7-cp37-abi3-macosx_10_9_universal2.whl (484 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.7 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.7 which is incompatible.
mediapipe-silicon 0.8.11 requires protobuf<4,>=3.11, but you have protobuf 4.21.7 which is incompatible.


In [1]:
%cd /Users/kimmonica/Documents/Smart_CCTV/LSTM

from basic_preprocessing import *
from deeplearning_check import *
from video_info import *
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import os

gpu_check()
set_global_determinism(seed=SEED)

/Users/kimmonica/Documents/Smart_CCTV/LSTM
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16453487497740499909
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 2985354354787009782
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]Metal device set to: 
tf 2.9.2
keras 2.9.0
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



objc[12901]: Class CaptureDelegate is implemented in both /Users/kimmonica/miniforge3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x28955a538) and /Users/kimmonica/miniforge3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x2b41c8860). One of the two will be used. Which one is undefined.
objc[12901]: Class CVWindow is implemented in both /Users/kimmonica/miniforge3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x28955a588) and /Users/kimmonica/miniforge3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x2ad4f0a68). One of the two will be used. Which one is undefined.
objc[12901]: Class CVView is implemented in both /Users/kimmonica/miniforge3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x28955a5b0) and /Users/kimmonica/miniforge3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x2ad4f0a90). One of the

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

### image

In [ ]:
frame = cv2.imread("/Users/kimmonica/Desktop/스크린샷 2022-10-13 오후 11.23.35.png", cv2.IMREAD_COLOR)
show(frame)
k = 'dd'
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    image, results = mediapipe_detection(frame, holistic)
    draw_styled_landmarks(image, results)
    show(image)
    print(results)
    draw_landmarks(frame, results)
    show(frame)
    print(len(results.pose_landmarks.landmark))
    pose = []
    for res in results.pose_landmarks.landmark:
        test = np.array([res.x, res.y, res.z, res.visibility])
        pose.append(test)
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    result_test = extract_keypoints(results)
    np.save('{}'.format(k), result_test)
    np.load('{}.npy'.format(k)) ### 좌표저장
print(result_test)

In [ ]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    file_path = "C:\\Users\\yukir\\Documents\\Monicas_workspace\\smart cctv\\1.배회\\"
    for i in file_list:
        print(i)

In [9]:
def get_duration(path):
    video = cv2.VideoCapture(path)
    du = video.get(cv2.CAP_PROP_POS_MSEC)
    return 

In [10]:
path = "C:\\Users\\yukir\\Documents\\Monicas_workspace\\smart cctv\\1.배회\\trim_C056300_018.mp4"
get_duration(path)

0.0

### video

In [4]:
folder_path = "/Users/kimmonica/Documents/Smart_CCTV/dataset/4.쓰러짐/"

file_list = []

for i in os.listdir(folder_path):
    if i[0]!='.':
        file_list.append(i)


In [5]:
round_folder_path = os.listdir("/Users/kimmonica/Documents/Smart_CCTV/LSTM/falling/")

round_file_list = []

for i in round_folder_path:
    folder_path = "/Users/kimmonica/Documents/Smart_CCTV/LSTM/falling/"
    new = folder_path+str(i)
    round_file_list.append(folder_path+i)

round_file_list[:2]

['/Users/kimmonica/Documents/Smart_CCTV/LSTM/falling/falling_21',
 '/Users/kimmonica/Documents/Smart_CCTV/LSTM/falling/falling_145']

In [15]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    file_path = "/Users/kimmonica/Documents/Smart_CCTV/dataset/4.쓰러짐/"
    vid_num = 0
    for num, i in enumerate(file_list[:3]):
        
        vid_path = file_path + i
        print(vid_path)
        cap, width, height, fps, fourcc, filename, out = vid_info(vid_path, 'XVID', '1013_test', 90)

        try:
            frame_num = 0
            while True:
                frame_num += 1
                try:
                    ret, frame = cap.read()
                    # blank_img = np.zeros((720, 1280, 3), np.uint8)
                    image, results = mediapipe_detection(frame, holistic)
                    draw_styled_landmarks(image, results)
                    # print(results)
                    draw_landmarks(image, results)
                    
                    # print(len(results.pose_landmarks.landmark))
                    pose = []
                    for res in results.pose_landmarks.landmark:
                        test = np.array([res.x, res.y, res.z, res.visibility])
                        pose.append(test)
                    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
                    result_test = extract_keypoints(results)
                    # frame_num +=1
                    npy_path = round_file_list[num]+'/{}'.format(str(frame_num))
                    np.save(npy_path, result_test)
                    if len(os.listdir(round_file_list[num]))>600:
                        break
                except:
                    pass
                cv2.imshow('frame', image)
                if cv2.waitKey(30) == 27:
                    break
        except:
            pass
    vid_num +=1
    lefted = len(file_list)- vid_num
    print('Lefted', lefted)
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

/Users/kimmonica/Documents/Smart_CCTV/dataset/4.쓰러짐/trim_C052303_006.mp4
< cv2.VideoCapture 0x2f1449710>
cap < cv2.VideoCapture 0x2f1449710>, width 1280.0, height 720.0, fps 90
codec {0} filename 1013_test.mp4


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


/Users/kimmonica/Documents/Smart_CCTV/dataset/4.쓰러짐/trim_C052303_012.mp4
< cv2.VideoCapture 0x2f61dced0>
cap < cv2.VideoCapture 0x2f61dced0>, width 1280.0, height 720.0, fps 90
codec {0} filename 1013_test.mp4


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


/Users/kimmonica/Documents/Smart_CCTV/dataset/4.쓰러짐/trim_C058203_001.mp4
< cv2.VideoCapture 0x2f16afd10>
cap < cv2.VideoCapture 0x2f16afd10>, width 1280.0, height 720.0, fps 90
codec {0} filename 1013_test.mp4


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Lefted 329


-1

In [13]:
len(os.listdir(round_file_list[0]))

127

In [ ]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
len(results.left_hand_landmarks.landmark)
